In [2]:
!pip install transformers > /dev/null
!pip install datasets > /dev/null

In [3]:
# -*- coding: utf-8 -*-
"""
Created on Sun Mar 10 16:27:03 2013

@author: Mohamed Aly <mohamed@mohamedaly.info>
"""

import codecs
import re

import numpy as np
import pandas as pd


class LABR:
    def __init__(self):
        self.REVIEWS_PATH = "data/labr/"
        self.RAW_REVIEWS_FILE = "raw_reviews.tsv"
        self.DELETED_REVIEWS_FILE = "deleted_reviews.tsv"
        self.CLEAN_REVIEWS_FILE = "reviews.tsv"

    # Copied from the PyArabic package.
    def arabicrange(self):
        """return a list of arabic characteres .
        Return a list of characteres between \u060c to \u0652
        @return: list of arabic characteres.
        @rtype: unicode;
        """

        mylist = []

        for i in range(0x0600, 0x00653):
            try:
                mylist.append(chr(i))
            except ValueError:
                pass

        return mylist

    # cleans a single review
    def clean_raw_review(self, body):
        # patterns to remove first
        pat = [ \
            (u'http[s]?://[a-zA-Z0-9_\-./~\?=%&]+', u''),  # remove links
            (u'www[a-zA-Z0-9_\-?=%&/.~]+', u''),
            #            u'\n+': u' ',                     # remove newlines
            (u'<br />', u' '),  # remove html line breaks
            (u'</?[^>]+>', u' '),  # remove html markup
            #            u'http': u'',
            (u'[a-zA-Z]+\.org', u''),
            (u'[a-zA-Z]+\.com', u''),
            (u'://', u''),
            (u'&[^;]+;', u' '),
            (u':D', u':)'),
            #            (u'[0-9/]+', u''),
            #            u'[a-zA-Z.]+': u'',
            #            u'[^0-9' + u''.join(self.arabicrange()) + \
            #                u"!.,;:$%&*%'#(){}~`\[\]/\\\\\"" + \
            #                u'\s^><\-_\u201D\u00AB=\u2026]+': u'',          # remove latin characters
            (u'\s+', u' '),  # remove spaces
            (u'\.+', u'.'),  # multiple dots
            (u'[\u201C\u201D]', u'"'),  # “
            (u'[\u2665\u2764]', u''),  # heart symbol
            (u'[\u00BB\u00AB]', u'"'),
            (u'\u2013', u'-'),  # dash
        ]

        # patterns that disqualify a review
        remove_if_there = [ \
            (u'[^0-9' + u''.join(self.arabicrange()) + \
             u"!.,;:$%&*%'#(){}~`\[\]/\\\\\"" + \
             u'\s\^><\-_\u201D\u00AB=\u2026+|' + \
             u'\u0660-\u066D\u201C\u201D' + \
             u'\ufefb\ufef7\ufef5\ufef9]+', u''),  # non arabic characters
        ]

        # patterns that disqualify if empty after removing
        remove_if_empty_after = [ \
            (u'[0-9a-zA-Z\-_]', u' '),  # alpha-numeric
            (u'[0-9' + u".,!;:$%&*%'#(){}~`\[\]/\\\\\"" + \
             u'\s\^><`\-=_+]+', u''),  # remove just punctuation
            (u'\s+', u' '),  # remove spaces
        ]

        # remove again
        # patterns to remove
        pat2 = [ \
            #            u'[^0-9' + u''.join(self.arabicrange()) + \
            #                u"!.,;:$%&*%'#(){}~`\[\]/\\\\\"" + \
            #                u'\s^><\-_\u201D\u00AB=\u2026]+': u'',          # remove latin characters
        ]

        skip = False

        # if empty body, skip
        if body == u'': skip = True

        # do some subsitutions
        for k, v in pat:
            body = re.sub(k, v, body)

        # remove if exist
        for k, v in remove_if_there:
            if re.search(k, body):
                skip = True

        # remove if empty after replacing
        for k, v in remove_if_empty_after:
            temp = re.sub(k, v, body)
            if temp == u" " or temp == u"":
                skip = True

        # do some more subsitutions
        if not skip:
            for k, v in pat2:
                body = re.sub(k, v, body)

        # if empty string, skip
        if body == u'' or body == u' ':
            skip = True

        if not skip:
            return body
        else:
            return u""

    # Read raw reviews from file and clean and write into clean_reviews
    def clean_raw_reviews(self):
        # input file
        in_file = codecs.open(self.REVIEWS_PATH + self.RAW_REVIEWS_FILE, 'r', encoding="utf-8")

        reviews = in_file.readlines()

        # Output file: rating<tab>content
        out_file = open(self.REVIEWS_PATH + self.CLEAN_REVIEWS_FILE, 'wb', buffering=100)

        deleted_file = open(self.REVIEWS_PATH + self.DELETED_REVIEWS_FILE, 'wb', buffering=100)

        counter = 1
        for i in range(0, len(reviews)):
            review = reviews[i]
            skip = False

            #           # If line starts with #, then skip
            #            if review[0] == u"#": continue

            # split by <tab>
            parts = review.split(u"\t")

            # rating is first part and body is last part
            rating = parts[0]
            review_id = parts[1]
            user_id = parts[2]
            book_id = parts[3]
            body = parts[4].strip()

            # clean body
            body = self.clean_raw_review(body)
            if body == u"": skip = True

            if i % 5000 == 0:
                print("review %d:" % i)

            # write output
            line = u"%s\t%s\t%s\t%s\t%s\n" % (rating, review_id, user_id, book_id, body)

            if not skip:
                out_file.write(line.encode('utf-8'))
                counter += 1
            else:
                deleted_file.write(line.encode('utf-8'))

    # Read the reviews file. Returns a tuple containing these lists:
    #   rating: the rating 1 -> 5
    #   review_id: the id of the review
    #   user_id: the id of the user
    #   book_id: the id of the book
    #   body: the text of the review
    def read_review_file(self, file_name):
        reviews = codecs.open(file_name, 'r', 'utf-8').readlines()

        # remove comment lines and newlines
        reviews = [r.strip() for r in reviews if r[0] != u'#']

        # parse
        rating = list()
        review_id = list()
        user_id = list()
        book_id = list()
        body = list()
        for review in reviews:
            # split by <tab>
            parts = review.split(u"\t")

            # rating is first part and body is last part
            rating.append(int(parts[0]))
            review_id.append(parts[1])
            user_id.append(parts[2])
            book_id.append(parts[3])
            if len(parts) > 4:
                body.append(parts[4])
            else:
                body.append(u"")

        return (rating, review_id, user_id, book_id, body)

    # Writes reviews to a file
    def write_review_file(self, file_name, rating, review_id, user_id,
                          book_id, body):

        lines = list()
        # loop
        for i in range(len(rating)):
            line = u"%s\t%s\t%s\t%s\t%s\n" % (rating[i], review_id[i],
                                              user_id[i], book_id[i],
                                              body[i])
            lines.append(line)

        open(file_name, 'w').write(u''.join(lines).encode('utf-8'))

    def read_clean_reviews(self):
        return self.read_review_file(self.REVIEWS_PATH + self.CLEAN_REVIEWS_FILE)

    def read_raw_reviews(self):
        return self.read_review_file(self.REVIEWS_PATH + self.RAW_REVIEWS_FILE)

    # Splits the dataset into a training/test sets in the setting of using 5
    # classes (predicting the rating value from 1 to 5)
    def split_train_test_5class(self, rating, percent_test,
                                balanced="unbalanced"):
        np.random.seed(1234)

        num_reviews = len(rating)
        review_ids = np.arange(0, num_reviews)

        if balanced == "unbalanced":
            ntest = np.floor(num_reviews * percent_test)
            np.random.shuffle(review_ids)

            test_ids = review_ids[:ntest]
            train_ids = review_ids[ntest:]

        elif balanced == "balanced":
            (sizes, bins) = np.histogram(rating, [1, 2, 3, 4, 5, 6])
            min_size = np.min(sizes)
            print(min_size)

            # sample review ids equally among classes
            test_ids = np.zeros((0,), dtype="int32")
            train_ids = np.zeros((0,), dtype="int32")
            rating = np.array(rating)
            ntest = np.floor(min_size * percent_test)
            for c in range(1, 6):
                cids = review_ids[np.nonzero(rating == c)]
                np.random.shuffle(cids)

                test_ids = np.r_[test_ids, cids[:ntest]]
                train_ids = np.r_[train_ids, cids[ntest:min_size]]

        train_file = self.REVIEWS_PATH + "5class-" + balanced + "-train.txt"
        test_file = self.REVIEWS_PATH + "5class-" + balanced + "-test.txt"

        open(train_file, 'w').write('\n'.join(map(str, train_ids)))
        open(test_file, 'w').write('\n'.join(map(str, test_ids)))

        return (train_ids, test_ids)

    # Splits the dataset into a training/test sets in the setting of using 2
    # classes (predicting the polarity of the review where ratings 1 & 2
    # are considered negative, ratings 4 & 5 are positive, and rating 3 is
    # ignored)
    def split_train_test_2class(self, rating, percent_test,
                                balanced="unbalanced"):
        np.random.seed(1234)

        rating = np.array(rating, dtype='int32')
        # length
        num_reviews = len(rating)
        review_ids = np.arange(0, num_reviews)

        # convert to binary, with ratings [1, 2] --> neg and [4, 5] --> pos
        rating[rating == 2] = 1
        rating[rating == 4] = 5

        ids = (rating == 1) + (rating == 5)
        review_ids = review_ids[ids]
        rating = rating[ids]
        rating[rating == 1] = 0
        rating[rating == 5] = 1

        # get length after filtering
        num_reviews = rating.shape[0]

        if balanced == "unbalanced":
            ntest = np.floor(num_reviews * percent_test)
            np.random.shuffle(review_ids)

            test_ids = review_ids[:ntest]
            train_ids = review_ids[ntest:]

        elif balanced == "balanced":
            (sizes, bins) = np.histogram(rating, [0, 1, 2])
            min_size = np.min(sizes)
            print("minsize for balanced " , min_size)

            # sample review ids equally among classes
            test_ids = np.zeros((0,), dtype="int32")
            train_ids = np.zeros((0,), dtype="int32")
            rating = np.array(rating)
            ntest = np.floor(min_size * percent_test)
            for c in [0, 1]:
                cids = review_ids[np.nonzero(rating == c)]
                np.random.shuffle(cids)

                test_ids = np.r_[test_ids, cids[:ntest]]
                train_ids = np.r_[train_ids, cids[ntest:min_size]]

        train_file = self.REVIEWS_PATH + "2class-" + balanced + "-train.txt"
        test_file = self.REVIEWS_PATH + "2class-" + balanced + "-test.txt"

        open(train_file, 'w').write('\n'.join(map(str, train_ids)))
        open(test_file, 'w').write('\n'.join(map(str, test_ids)))

        return (train_ids, test_ids)

    # Reads a training or test file. The file contains the indices of the
    # reviews from the clean reviews file.
    def read_train_test_file(self, file_name):
        ins = open(file_name).readlines()
        ins = [int(i.strip()) for i in ins]

        return ins

    # A helpter function.
    def set_binary_klass(self, ar):
        ar[(ar == 1) + (ar == 2)] = 0
        ar[(ar == 4) + (ar == 5)] = 1

    # Returns (train_x, train_y, test_x, test_y)
    # where x is the review body and y is the rating (1->5 or 0->1)
    def get_train_test(self, klass="2", balanced="balanced"):
        (rating, a, b, c, body) = self.read_clean_reviews()
        rating = np.array(rating)
        body = pd.Series(body)

        train_file = (self.REVIEWS_PATH + klass + "class-" + balanced + "-train.txt")
        test_file = (self.REVIEWS_PATH + klass + "class-" + balanced + "-test.txt")

        train_ids = self.read_train_test_file(train_file)
        test_ids = self.read_train_test_file(test_file)

        train_y = rating[train_ids]
        test_y = rating[test_ids]
        train_x = body[train_ids]
        test_x = body[test_ids]

        if klass == "2":
            self.set_binary_klass(train_y)
            self.set_binary_klass(test_y)

        return (train_x, train_y, test_x, test_y)

In [4]:
from datasets import load_dataset, load_metric, load_dataset

In [5]:
import json

labr = LABR()

datas = [
    dict(name="2-balanced", params=dict(klass="2", balanced="balanced")),
    dict(name="2-unbalanced", params=dict(klass="2", balanced="unbalanced"))
]

# Load the data
print(60 * "-")
data = datas[1]
print("Loading data:", data['name'])
(d_train, y_train, d_test, y_test) = labr.get_train_test(**data['params'])

------------------------------------------------------------
Loading data: 2-unbalanced


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(d_train, y_train, test_size=0.2, random_state=43, stratify=y_train)

#df train
frame_train = { 'text': train_X, 'label': train_y}
df_train = pd.DataFrame(frame_train)
df_train.to_csv("data/labr/train.csv", encoding='utf-8', index=False)

# df val
frame_val = { 'text': val_X, 'label': val_y}
df_val = pd.DataFrame(frame_val)
df_val.to_csv("data//labr/val.csv", encoding='utf-8', index=False)


#df test
frame_test = { 'text': d_test, 'label': y_test }
df_test = pd.DataFrame(frame_test)
df_test.to_csv("data/labr/test.csv", encoding='utf-8', index=False)

In [7]:
df_test.shape

(10211, 2)

In [8]:
ds = load_dataset('csv', data_files={'train': 'data/labr/train.csv', 'val': 'data/labr/val.csv', 'test': 'data/labr/test.csv'})
ds

Using custom data configuration default-9bf6b5f3fa9f7ded


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-9bf6b5f3fa9f7ded/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 32676
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 8169
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 10211
    })
})

In [10]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model_path = "mofawzy/bert-labr-unbalanced"
model_testing = AutoModelForSequenceClassification.from_pretrained(model_path,num_labels=2)
tokenizer_testing = AutoTokenizer.from_pretrained(model_path)

Downloading:   0%|          | 0.00/799 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/381 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/326k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/818k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [11]:
def tokenize(examples):
    outputs = tokenizer_testing(examples['text'], truncation=True,max_length=64)
    return outputs

tokenized_ds = ds.map(tokenize, batched=True)

  0%|          | 0/33 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

# Test the fine-tuned model

In [12]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
test_trainer = Trainer(model=model_testing,tokenizer=tokenizer_testing)

In [13]:
raw_pred, _, _ = test_trainer.predict(tokenized_ds['test']) 

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 10211
  Batch size = 8


In [14]:
# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)

In [15]:
len(y_pred), type(y_pred)

(10211, numpy.ndarray)

In [16]:
len(y_test), type(y_test)

(10211, numpy.ndarray)

In [17]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred,digits=4))

              precision    recall  f1-score   support

           0     0.8109    0.6832    0.7416      1670
           1     0.9399    0.9689    0.9542      8541

    accuracy                         0.9221     10211
   macro avg     0.8754    0.8260    0.8479     10211
weighted avg     0.9188    0.9221    0.9194     10211

